<a href="https://colab.research.google.com/github/ollihansen90/Mathe-SH/blob/main/Schachcomputer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
bauer = 1 # 1
springer = 2 # 3
laeufer = 3 # 3
turm = 4 # 5
dame = 5 # 9
koenig = 6 # 100
leer = 0

schachbrett = [
    [-turm, -springer, -laeufer, -dame, -koenig, -laeufer, -springer, -turm],
    [-bauer, -bauer, -bauer, -bauer, -bauer, -bauer, -bauer, -bauer],
    [leer, leer, leer, leer, leer, leer, leer, leer],
    [leer, leer, leer, leer, leer, leer, leer, leer],
    [leer, leer, leer, leer, leer, leer, leer, leer],
    [leer, leer, leer, leer, leer, leer, leer, leer],
    [bauer, bauer, bauer, bauer, bauer, bauer, bauer, bauer],
    [turm, springer, laeufer, dame, koenig, laeufer, springer, turm]
]

print(schachbrett)
figuren = ["\x1b[97m\u2659\x1b[30m", "\u2659","\u2658","\u2657","\u2656","\u2655","\u2654","\u265A","\u265B","\u265C","\u265D","\u265E","\u265F"]
print("leer" ,figuren[-leer])
print("bauer" ,figuren[-bauer])
print("springer" ,figuren[-springer])
print("laeufer" ,figuren[-laeufer])
print("turm" ,figuren[-turm])
print("dame" ,figuren[-dame])
print("koenig" ,figuren[-koenig])

def print_brett(schachbrett):
    output = ''
    bg = ["\x1b[47m", "\x1b[107m"]
    counter = 0
    for reihe in schachbrett:
        counter += 1
        for feld in reihe:
            output += bg[counter%2]+"\x1b[8m"+figuren[feld]
            counter += 1
        output += '\n'
    print(output)

print_brett(schachbrett)


[[-4, -2, -3, -5, -6, -3, -2, -4], [-1, -1, -1, -1, -1, -1, -1, -1], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1], [4, 2, 3, 5, 6, 3, 2, 4]]
leer ♙
bauer ♟
springer ♞
laeufer ♝
turm ♜
dame ♛
koenig ♚
♜♞♝♛♚♝♞♜
♟♟♟♟♟♟♟♟
♙♙♙♙♙♙♙♙
♙♙♙♙♙♙♙♙
♙♙♙♙♙♙♙♙
♙♙♙♙♙♙♙♙
♙♙♙♙♙♙♙♙
♖♘♗♕♔♗♘♖



In [26]:
schachbrett_fig = [[figuren[fig] for fig in lis] for lis in schachbrett]
print(*schachbrett_fig, sep="\n")

['♜', '♞', '♝', '♛', '♚', '♝', '♞', '♜']
['♟', '♟', '♟', '♟', '♟', '♟', '♟', '♟']
['  ', '  ', '  ', '  ', '  ', '  ', '  ', '  ']
['  ', '  ', '  ', '  ', '  ', '  ', '  ', '  ']
['  ', '  ', '  ', '  ', '  ', '  ', '  ', '  ']
['  ', '  ', '  ', '  ', '  ', '  ', '  ', '  ']
['♙', '♙', '♙', '♙', '♙', '♙', '♙', '♙']
['♖', '♘', '♗', '♕', '♔', '♗', '♘', '♖']


In [44]:
!python3 --version

Python 3.10.12
